### Initialization

In [ ]:
# For Colab only!

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
import torch
from torch.nn import functional as F
from torchvision import datasets, transforms
from torch import nn

In [3]:
print(tf.__version__)
print(tf.test.is_gpu_available())

2.1.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


In [4]:
print(torch.__version__)
print(torch.cuda.is_available())

1.4.0
True


### Data Loading
MINST data set

In [19]:
import numpy as np

batch_size=32
learning_rate=0.01
epochs=10

In [20]:
(x, y),(x_test, y_test) = keras.datasets.mnist.load_data()

ds_train = tf.data.Dataset.from_tensor_slices((x,y))
ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def preprocess(x, y):
  x = (tf.cast(x, tf.float32)/255)-0.1307
  y = tf.cast(y, tf.int32)
#   y = tf.one_hot(y,depth=10)   
  return x, y

ds_train = ds_train.map(preprocess).shuffle(1000).batch(batch_size)
ds_test = ds_test.map(preprocess).shuffle(1000).batch(batch_size)


In [21]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True)

In [22]:
print(type(ds_test))
image, label = next(iter(ds_test))
print(image.shape, label.shape)

<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
(32, 28, 28) (32,)


In [23]:
print(type(train_loader))
[image, label] = next(iter(train_loader))
print(image.shape, label.shape)

<class 'torch.utils.data.dataloader.DataLoader'>
torch.Size([32, 1, 28, 28]) torch.Size([32])


### CNN




Tensorflow

[b, 28, 28 1]  with kernel [3, 1, 3, 3] padding = "same", stride = 1 -> [b, 14, 14, 3]

> NOTE:
>
> Here In `layers.Conv2D` parameters: filters = 3, kernel_size = (3,3), strides = (1,1)

[b, 14, 14, 3] -> maxpooling -> [b, 7, 7, 3]

[b, 14, 14, 3]  with kernel [6, 3, 3, 3] padding = "same", stride = 2 -> [b, 7, 7, 6]

[b, 7, 7, 6]  flatten() -> [b, 7*7*6]

[b, 7*7*6] -> Dense -> [b, 10]


In [ ]:
class CNN_model(keras.Model):
    def __init__(self):
        super().__init__()
    
        self.model = keras.Sequential(
            [layers.Conv2D(filters=3, kernel_size=(3,3), strides=(1,1),padding="same"),
            layers.MaxPool2D(pool_size=(2,2)),
            layers.ReLU(),
            layers.Conv2D(6,(3,3),(2,2),"same"),
            layers.ReLU(),
            layers.Flatten(),
            layers.Dense(10)]
            )
    
    def call(self,x):
        x = self.model(x)
        
        return x
    
model = CNN_model()
model.build(input_shape = (None,28,28,1))
model.summary()
optimizer = tf.optimizers.Adam(learning_rate)
    
for epoch in range(epochs):
    
    for step, (x, y) in enumerate(ds_train):
        x = tf.reshape(x, [-1, 28,28,1])
        with tf.GradientTape() as tape:            
            logits = model(x)
            
            losses = tf.losses.sparse_categorical_crossentropy(y,logits,from_logits=True)
            loss = tf.reduce_mean(losses)
            
        grads = tape.gradient(loss, model.variables)
        
        optimizer.apply_gradients(zip(grads, model.variables))
        
        if(step%100==0):
            print("epoch:{}, step:{} loss:{}".
                  format(epoch, step, loss.numpy()))
            
            
#             test accuracy: 
            total_correct = 0
            total_num = 0
            
            for x_test, y_test in ds_test:
                x_test = tf.reshape(x_test, [-1, 28,28,1])
                y_pred = tf.argmax(model(x_test),axis=1)
                y_pred = tf.cast(y_pred, tf.int32)
                correct = tf.cast((y_pred == y_test), tf.int32)
                correct = tf.reduce_sum(correct)
                
                total_correct += int(correct)
                total_num += x_test.shape[0]
        
            
            accuracy = total_correct/total_num
            print('accuracy: ', accuracy)


Model: "cnn_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    multiple                  3148      
Total params: 3,148
Trainable params: 3,148
Non-trainable params: 0
_________________________________________________________________
epoch:0, step:0 loss:2.328885078430176
accuracy:  0.1409
epoch:0, step:100 loss:0.14413821697235107
accuracy:  0.8648
epoch:0, step:200 loss:0.28817737102508545
accuracy:  0.9059
epoch:0, step:300 loss:0.17282797396183014
accuracy:  0.9134
epoch:0, step:400 loss:0.0806659460067749
accuracy:  0.9312
epoch:0, step:500 loss:0.06717875599861145
accuracy:  0.9398
epoch:0, step:600 loss:0.2847217917442322
accuracy:  0.9508
epoch:0, step:700 loss:0.22499491274356842
accuracy:  0.9481
epoch:0, step:800 loss:0.16353292763233185
accuracy:  0.9512
epoch:0, step:900 loss:0.06671904027462006
accuracy:  0.9484
epoch:0, step:1000 loss:0.09817676246166229

PyTorch

[b, 1, 28, 28]  with kernel [3, 1, 3, 3] padding = 1, stride = 1 -> [b, 3, 28, 28]

> NOTE:
>
> Here In `nn.Conv2d` parameters: filters = 3, kernel_size = (3,3), strides = (2,2)

[b, 3, 28, 28] with `nn.MaxPool2d(kernel_size=2)` -> [b, 3, 14 ,14 ] 

[b, 3, 7, 7]  with kernel [6, 3, 3, 3] padding = 1, stride = 2 -> [b, 6, 7, 7]

[b, 6, 7, 7]  flatten() -> [b, 6 * 7 * 7]

[b, 6 * 7 * 7] -> Dense -> [b, 10]

In [18]:
class CNN_NN(nn.Module):
    def __init__(self):
        super().__init__()
    
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=3,kernel_size=3,stride=1,padding=1),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(3,6,kernel_size=3,stride=2,padding=1),
            nn.ReLU(inplace=True),
            nn.Flatten(),
            nn.Linear(6*7*7,10)
            )
    
    def forward(self, x):
        x = self.model(x)
        
        return x
device = torch.device('cuda:0')

network = CNN_NN().to(device)        
optimizer = torch.optim.Adam(network.parameters(),
                            lr=learning_rate)
criteon = torch.nn.CrossEntropyLoss().to(device)

for epoch in range(epochs):
    
    for step, (x, y) in enumerate(train_loader):
        x = x.reshape(-1,1,28,28)
        
        x, y = x.to(device), y.to(device)
        
        logits = network(x)
        loss = criteon(logits, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(step%100 == 0):
            print("epoch:{}, step:{}, loss:{}".
                  format(epoch, step, loss.item()))
        
#             test accuracy
            total_correct = 0
            total_num = 0    

            for x_test, y_test in test_loader:
                    x_test = x_test.reshape(-1,1,28,28)
                    x_test, y_test = x_test.to(device), y_test.to(device)

                    y_pred = network(x_test)
                    y_pred = torch.argmax(y_pred, dim = 1)
                    correct = y_pred == y_test
                    correct = correct.sum()

                    total_correct += correct
                    total_num += x_test.shape[0]

            acc = total_correct.float()/total_num
            print("accuracy: ", acc.item())
                
                

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 7.93 GiB total capacity; 0 bytes already allocated; 24.50 MiB free; 0 bytes reserved in total by PyTorch)

In [ ]:
# OUT SIZE TEST BLOCK FOR PYTORCH

x = torch.rand(1,1,28,28)
layer1 = nn.Conv2d(1,3,kernel_size=3,stride=1,padding=1)
layer2 = nn.MaxPool2d(kernel_size=2)
layer3 = nn.Conv2d(3,6,kernel_size=3,stride=2,padding=1)

out = layer1(x)
print(out.shape)

out = layer2(out)

print(out.shape)

out = layer3(out)
print(out.shape)